## Sentiment Analysis using Sequential model

In [90]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing

samples = ['너 오늘 이뻐 보인다',
           '나는 오늘 기분이 더러워',
           '끝내주는데, 좋은 일이 있나봐',
           '나 좋은 일이 생겼어',
           '아 오늘 진짜 짜증나',
           '환상적인데, 정말 좋은거 같아']

labels = [[1],[0],[1],[1],[0],[1]]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)

word_idx = tokenizer.word_index

In [91]:
print(sequences[0])
print(word_idx)

[4, 1, 5, 6]
{'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}


In [92]:
batch_size = 2
num_epochs = 100
vocab_size = len(word_idx) + 1 # padding token 때문에 + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

### Method 1. Sequential API

In [93]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length=4))
model.add(layers.Lambda(lambda x:tf.reduce_mean(x, axis=1))) # reduce_mean은 차원을 감소하면서 평균을 계산
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension, activation='sigmoid'))

### Method 2. Sequential API + list 형식

In [94]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length=4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)),
    layers.Dense(hidden_dimension, activation='relu'),
    layers.Dense(output_dimension, activation='sigmoid')
])

### Method 3. Functional API

In [95]:
inputs = layers.Input(shape = (4,))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 1s 13ms/step - loss: 0.6953 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6766 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6627 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6504 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6346 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6171 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5979 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5732 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5433 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 6ms/step - loss: 0.5093 - accuracy: 1.0000
Epoch 11/100
3/3 [========

### Method 4. Subclassing (Custom model)

In [96]:
class CustomModel(tf.keras.Model):

  def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
    super(CustomModel, self).__init__(name='my_model') # name을 지정할 때는 공백이 포함되지 않도록 조심
    self.embedding = layers.Embedding(vocab_size, embed_dimension)
    self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
    self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

  def call(self, inputs):
    x = self.embedding(inputs)
    x = tf.reduce_mean(x, axis=1)
    x = self.dense_layer(x)
    x = self.output_layer(x)

    return x

In [97]:
model = CustomModel(vocab_size=vocab_size,
                    embed_dimension=emb_size,
                    hidden_dimension=hidden_dimension,
                    output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 1s 8ms/step - loss: 0.6884 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 9ms/step - loss: 0.6696 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6529 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6348 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 0.6137 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5927 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5647 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 9ms/step - loss: 0.5327 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 0.4968 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 9ms/step - loss: 0.4564 - accuracy: 1.0000
Epoch 11/100
3/3 [=========

## Feature extraction using sklearn

### CountVectorizer

- text data에서 횟수를 기준으로 feature를 extract하는 방법
- 어떤 단위의 횟수를 셀 것인지는 선택사항
- 단위는 단어가 될 수도 있고, 문자 하나하나가 될 수 있음
- **But. 조사나 지시대명사가 높은 특징 값을 가지기에 유의미하게 사용되기 어려움**

In [98]:
from sklearn.feature_extraction.text import CountVectorizer

In [99]:
text_data = ['나는 배가 고프다',
             '내일 점심 뭐먹지',
             '내일 공부 해야겠다',
             '점심 먹고 공부해야지']

count_vectorizer = CountVectorizer()

In [100]:
count_vectorizer.fit(text_data)
print(count_vectorizer.vocabulary_)

{'나는': 3, '배가': 7, '고프다': 0, '내일': 4, '점심': 8, '뭐먹지': 6, '공부': 1, '해야겠다': 9, '먹고': 5, '공부해야지': 2}


In [101]:
sentence = [text_data[0]]
print(count_vectorizer.transform(sentence).toarray())

[[1 0 0 1 0 0 0 1 0 0]]


### TfidfVectorizer

- TF-IDF라는 특정한 값을 사용해서 text data의 feature를 extract
- TF(Term Frequency): 특정 단어가 하나의 데이터 안에서 등장하는 횟수
- DF(Document Frequency): 문서 빈도 값. 특정 단어가 여러 데이터에 자주 등장하는지를 알려주는 지표
- IDF(Inverse Document Frequency): 1/DF, 특정 단어가 다른 데이터에 등장하지 않을수록 값이 커짐
- TF-IDF: TF * IDF. 즉, 어떤 단어가 해당 문서에서 자주 등장하지만 다른 문서에서는 많이 없는 단어일수록 높은 값을 가짐

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [103]:
text_data = ['나는 배가 고프다',
             '내일 점심 뭐먹지',
             '내일 공부 해야겠다',
             '점심 먹고 공부해야지']

tfidf_vectorizer = TfidfVectorizer()

In [104]:
tfidf_vectorizer.fit(text_data)
print(tfidf_vectorizer.vocabulary_)

{'나는': 3, '배가': 7, '고프다': 0, '내일': 4, '점심': 8, '뭐먹지': 6, '공부': 1, '해야겠다': 9, '먹고': 5, '공부해야지': 2}


In [105]:
sentence = [text_data[2]]
print(tfidf_vectorizer.transform(sentence).toarray())

[[0.         0.61761437 0.         0.         0.48693426 0.
  0.         0.         0.         0.61761437]]


## Tokenizing

### English tokenizing using NLTK

In [106]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

#### 단어 단위 토크나이징

In [107]:
from nltk.tokenize import word_tokenize

sentence = 'Natural language processing (NLP) is a subfield of computer science, information engineering, \
and artificial intelligence concerned with the interactions between computers and human (natural) languages, \
in particular how to program computers to process and analyze large amounts of natural language data.'

print(word_tokenize(sentence))

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'information', 'engineering', ',', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.']


#### 문장 단위 토크나이징

In [108]:
from nltk.tokenize import sent_tokenize

sentence = 'Natural language processing (NLP) is a subfield of computer science, information engineering, \
and artificial intelligence concerned with the interactions between computers and human (natural) languages, \
in particular how to program computers to process and analyze large amounts of natural language data. \
Challenges in natural language processing frequently involve speech recognition, natural language understanding, \
and natural language generation'

print(sent_tokenize(sentence))

['Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.', 'Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation']


### English tokenizing using Spacy

In [109]:
!python -m spacy download en

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-05 09:04:12.249765: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 09:04:12.250079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-05 09:04:12.250114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [110]:
import spacy

nlp = spacy.load('en_core_web_sm')

sentence = 'Natural language processing (NLP) is a subfield of computer science, information engineering, \
and artificial intelligence concerned with the interactions between computers and human (natural) languages, \
in particular how to program computers to process and analyze large amounts of natural language data. \
Challenges in natural language processing frequently involve speech recognition, natural language understanding, \
and natural language generation'

doc = nlp(sentence)

In [111]:
word_tokenized_sentence = [token.text for token in doc]
sentence_tokenized_list = [sent.text for sent in doc.sents]
print(word_tokenized_sentence)
print(sentence_tokenized_list)

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'information', 'engineering', ',', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.', 'Challenges', 'in', 'natural', 'language', 'processing', 'frequently', 'involve', 'speech', 'recognition', ',', 'natural', 'language', 'understanding', ',', 'and', 'natural', 'language', 'generation']
['Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.', 'Challenges in natural langua

In [112]:
# list comprehension
word_tokenized_sentence = [token.text for token in doc]
sentence_tokenized_list = [sent.text for sent in doc.sents]
print(word_tokenized_sentence)
print(sentence_tokenized_list)

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', ',', 'information', 'engineering', ',', 'and', 'artificial', 'intelligence', 'concerned', 'with', 'the', 'interactions', 'between', 'computers', 'and', 'human', '(', 'natural', ')', 'languages', ',', 'in', 'particular', 'how', 'to', 'program', 'computers', 'to', 'process', 'and', 'analyze', 'large', 'amounts', 'of', 'natural', 'language', 'data', '.', 'Challenges', 'in', 'natural', 'language', 'processing', 'frequently', 'involve', 'speech', 'recognition', ',', 'natural', 'language', 'understanding', ',', 'and', 'natural', 'language', 'generation']
['Natural language processing (NLP) is a subfield of computer science, information engineering, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.', 'Challenges in natural langua

### Korean tokenizing using KoNLPy

- KoNLPy
  - 한글 자연어 처리를 쉽고 간결하게 처리할 수 있도록 만들어진 오픈소스 라이브러리
  - 국내에 이미 만들어져 사용되고 있는 여러 형태소 분석기를 사용할 수 있게 허용

In [113]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InR

In [114]:
import konlpy

In [115]:
from konlpy.tag import Okt

okt = Okt()

- **Okt()**

  - `okt.morphs()`: 텍스트를 형태소 단위로 나눈다. 옵션으로는 norm과 stem이 있다. 각각 `True` 혹은 `False` 값을 받으며, norm은 normalize의 약자로서 문장을 정규화하는 역할을 하고, stem은 각 단어에서 어간을 추출하는 기능이다. 각각 `True`로 설정하면 각 기능이 적용된다. 옵션을 지정하지 않으면 기본값은 둘 다 `False`로 설정된다.

  - `okt.nouns()`: 텍스트에서 명사만 뽑아낸다.

  - `okt.phrases()`: 텍스트에서 어절을 뽑아낸다.
  
  - `okt.pos()`: 위의 세 함수는 어간/명사/어절 등을 추출해내는 추출기로 동작했다면 pos 함수는 각 품사를 태깅하는 역할을 한다. 품사를 태깅한다는 것은 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함께 리스트화하는 것을 의미한다. 이 함수에서도 옵션을 설정할 수 있는데, morphs 함수와 마찬가지로 norm, stem 옵션이 있고 추가적으로 join 함수가 있는데 이 옵션 값을 `True`로 설정하면 나눠진 형태소와 품사를 '형태소/품사'형태로 같이 붙여서 리스트화한다.

In [116]:
text = '한글 자연어 처리는 재밌다 이제부터 열심히 해야지ㅎㅎㅎ'

print(okt.morphs(text))
print(okt.morphs(text, stem=True))
print(okt.morphs(text, norm=True))
print(okt.morphs(text, stem=True, norm=True))

['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '하다', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '해야지', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '이제', '부터', '열심히', '하다', 'ㅎㅎㅎ']


In [117]:
print(okt.nouns(text))
print(okt.phrases(text))

['한글', '자연어', '처리', '이제']
['한글', '한글 자연어', '한글 자연어 처리', '이제', '자연어', '처리']


In [118]:
print(okt.pos(text))
print(okt.pos(text, join=True))

[('한글', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재밌다', 'Adjective'), ('이제', 'Noun'), ('부터', 'Josa'), ('열심히', 'Adverb'), ('해야지', 'Verb'), ('ㅎㅎㅎ', 'KoreanParticle')]
['한글/Noun', '자연어/Noun', '처리/Noun', '는/Josa', '재밌다/Adjective', '이제/Noun', '부터/Josa', '열심히/Adverb', '해야지/Verb', 'ㅎㅎㅎ/KoreanParticle']


#### KoNLPy Data

In [119]:
from konlpy.corpus import kolaw
from konlpy.corpus import kobill

In [120]:
kolaw.open('constitution.txt').read()[:20]

'대한민국헌법\n\n유구한 역사와 전통에 '

In [121]:
kobill.open('1809890.txt').read()

'지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.  11.  12.  \n\n발  의  자 : 정의화․이명수․김을동 \n\n이사철․여상규․안규백\n\n황영철․박영아․김정훈\n\n김학송 의원(10인)\n\n제안이유 및 주요내용\n\n  초등학교 저학년의 경우에도 부모의 따뜻한 사랑과 보살핌이 필요\n\n한 나이이나, 현재 공무원이 자녀를 양육하기 위하여 육아휴직을 할 \n\n수 있는 자녀의 나이는 만 6세 이하로 되어 있어 초등학교 저학년인 \n\n자녀를 돌보기 위해서는 해당 부모님은 일자리를 그만 두어야 하고 \n\n이는 곧 출산의욕을 저하시키는 문제로 이어질 수 있을 것임.\n\n  따라서 육아휴직이 가능한 자녀의 연령을 만 8세 이하로 개정하려\n\n는 것임(안 제63조제2항제4호).\n\n- 1 -\n\n\x0c법률  제        호\n\n지방공무원법 일부개정법률안\n\n지방공무원법 일부를 다음과 같이 개정한다.\n\n제63조제2항제4호 중 “만 6세 이하의 초등학교 취학 전 자녀를”을 “만 \n\n8세 이하(취학 중인 경우에는 초등학교 2학년 이하를 말한다)의 자녀를”\n\n로 한다.\n\n부      칙\n\n이 법은 공포한 날부터 시행한다.\n\n- 3 -\n\n\x0c신 ·구조문대비표\n\n현      행\n\n개   정   안\n\n제63조(휴직) ① (생  략)\n\n제63조(휴직) ① (현행과 같음)\n\n  ② 공무원이 다음 각 호의 어\n\n  ② -------------------------\n\n느 하나에 해당하는 사유로 휴\n\n----------------------------\n\n직을 원하면 임용권자는 휴직\n\n----------------------------\n\n을 명할 수 있다. 다만, 제4호\n\n-------------.---------------\n\n의 경우에는 대통령령으로 정\n\n---------------------------

#### re

  - re는 파이썬 정규 표현식(Regular Expression) 라이브러리
  - 문자열을 쉽게 다룰 수 있기에 전처리 과정에서 많이 사용
  - 특히 문자열을 제거하거나 검색 혹은 치환 등에 주로 사용

Python regular expression

|기호|의미|
|---|---|
|.|줄 바꿈을 제외한 모든 문자 |
|^|문자열의 시작|
|$|문자열의 끝|
|*|앞에 있는 문자가 0회 이상 반복된 문자열|
|+|앞에 있는 문자가 1회 이상 반복된 문자열|
|{m}|앞 문자를 m회 반복하는 문자열|
|{m,n}|앞 문자를 m~n회 반복하는 문자열|
|?|앞 문자가 나오거나 나오지 않는 문자열 ({0,1}와 동일)|
|\d|숫자|
|\D|숫자가 아닌 문자|
|\w|숫자 혹은 문자|
|\W|숫자 혹은 문자가 아닌 것|
|(...)|괄호 안의 모든 정규 표현식을 만족하는 문자|
|[a,b,c]|a,b,c 중 한 개의 문자와 일치|

In [122]:
import re

pattern = '\W+'
re_pattern = re.compile(pattern)

In [123]:
re.search("(\w+)", "wow, it is awesome")

<re.Match object; span=(0, 3), match='wow'>

In [124]:
re.split("\W", "wow, it is awesome")

['wow', '', 'it', 'is', 'awesome']

In [125]:
re.sub("\d", "number", "7 candy")

'number candy'

#### Beautiful Soup

- 주로 웹 크롤링에 사용되는 라이브러리
- HTML 문서 혹은 XML 문서에서 데이터를 불러오는 데 사용
- 주로 HTML 태그를 제거하기 위해 사용

In [126]:
from bs4 import BeautifulSoup

In [127]:
text = '<body> 이 글은 Beautiful soup 라이브러리를 사용하는 방법에 대한 글입니다.<br> </br>\
라이브러리를 사용하면 쉽게 HTML 태그를 제거할 수 있습니다.'

text = BeautifulSoup(text, "html5lib").get_text()

print(text)

 이 글은 Beautiful soup 라이브러리를 사용하는 방법에 대한 글입니다. 라이브러리를 사용하면 쉽게 HTML 태그를 제거할 수 있습니다.
